In [105]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the original data
original_data = pd.read_csv('data/car-price.csv')

In [106]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np

def preprocess(data):
    
    # Remove 'Model' and 'ID' columns
    data = data.drop(['Model', 'ID'], axis=1)
    
    # Replace '-' with NaN in 'Levy'
    data['Levy'].replace('-', np.nan, inplace=True)
    # Convert 'Levy' to numerical type
    data['Levy'] = pd.to_numeric(data['Levy'])  
    # Convert 'Mileage' to numerical type
    if data['Mileage'].dtype == 'object':
        data['Mileage'] = data['Mileage'].str.replace(' km', '')
    data['Mileage'] = pd.to_numeric(data['Mileage']) 
    
    outlier_columns = ['Price','Levy', 'Mileage']

    # Remove outliers
    for col in outlier_columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]


    # Extract 'Turbo' information and convert 'Engine volume' to numerical type
    data['Turbo'] = data['Engine volume'].apply(lambda x: 1 if isinstance(x, str) and 'Turbo' in x else 0)
    if data['Engine volume'].dtype == 'object':
        data['Engine volume'] = data['Engine volume'].str.replace(' Turbo', '')
    data['Engine volume'] = pd.to_numeric(data['Engine volume'])
    
    # Leather interior convert to boolean
    data['Leather interior'] = data['Leather interior'].map({'Yes': 1, 'No': 0})

    # Map the 'Doors' column
    doors_mapping = {
        '04-May': '4-5',
        '02-Mar': '2-3',
        '>5': '>5'
    }
    data['Doors'] = data['Doors'].map(doors_mapping)
        
    top_manufacturers = data['Manufacturer'].value_counts().nlargest(10).index
    data['Manufacturer'] = data['Manufacturer'].apply(lambda x: x if x in top_manufacturers else 'Other')
    data['Manufacturer'] = data['Manufacturer'].str.title()
    top_categories = data['Category'].value_counts().nlargest(5).index
    data['Category'] = data['Category'].apply(lambda x: x if x in top_categories else 'Other')
    top_colors = data['Color'].value_counts().nlargest(5).index
    data['Color'] = data['Color'].apply(lambda x: x if x in top_colors else 'Other')
    top_fuel_types = data['Fuel type'].value_counts().nlargest(3).index
    data.loc[~data['Fuel type'].isin(top_fuel_types), 'Fuel type'] = 'Other'
    
    # Get a list of all column names
#     cols = list(data.columns)
#     cols.remove('Price')
#     cols.append('Price')
#     data = data[cols]
    
#     rename_cols = {"Levy":"levy", "Manufacturer": "manufacturer",  'Prod. year': "year", 
#               'Category': 'category', 'Leather interior': "leather_interior",'Fuel type':'fuel_type',
#               'Engine volume':'engine_volume', 'Mileage': 'mileage', 'Cylinders': 'cylinders',
#               'Gear box type': 'gear_box_type', 'Drive wheels':'drive_wheels', 'Doors':'doors',
#               "Wheel":'wheel', 'Color':'color', 'Airbags': 'airbags', 'Turbo': "turbo", 'Price': 'price'}
    
#     data = data.rename(columns = rename_cols)
    
#     data.to_csv("data/data.csv")
    
#     return data

#     Fill missing values

    num_cols = data.select_dtypes(exclude="object").columns[1:]
    cat_cols = data.select_dtypes(include="object").columns
    
    imputer = SimpleImputer(strategy='median')
    for col in num_cols:
        data[col] = imputer.fit_transform(data[col].values.reshape(-1, 1))
        
    # Separate features and target
    X = data.drop('Price', axis=1)
    y = data['Price']

#     # Apply StandardScaler to numerical columns and OneHotEncoder to categorical columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(), cat_cols)
        ])
    X_transformed = preprocessor.fit_transform(X)
    
#     # Get the feature names after one-hot encoding
    feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(input_features=cat_cols)
    feature_names = np.concatenate([num_cols, feature_names])
    
    return X_transformed, y, feature_names


In [97]:
# cleaned_data = preprocess(original_data)
# cleaned_data

In [98]:
# cat_cols = cleaned_data.select_dtypes(include="object").columns

# for col in cat_cols:
#     print(col.lower(), list(cleaned_data[col].unique()))

In [107]:
# Assume that 'new_data' is a DataFrame containing your new data
X,y,features = preprocess(original_data)

In [108]:
len(features)

47

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the models
model_rf = RandomForestRegressor(random_state=42)
model_cb = CatBoostRegressor(random_seed=42, verbose=0)  # verbose=0 to turn off training output
model_xgb = XGBRegressor(random_state=42)

# Train the models
model_rf.fit(X_train, y_train)
model_cb.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)

# Use the models to make predictions on the testing data
y_pred_rf = model_rf.predict(X_test)
y_pred_cb = model_cb.predict(X_test)
y_pred_xgb = model_xgb.predict(X_test)

# Combine the predictions
y_pred_combined = (y_pred_rf + y_pred_cb + y_pred_xgb) / 3

# Calculate and print the root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred_combined))
print(f'Root Mean Squared Error: {rmse}')

# Calculate and print the R-squared value
r2 = r2_score(y_test, y_pred_combined)
print(f'R-squared: {r2}')

Root Mean Squared Error: 5277.115461257566
R-squared: 0.806968877980063
